# Step-by-step run of alphazero self-play & training.


In [ ]:
import os
import time
from pathlib import Path
from collections import defaultdict, Counter
import asyncio
from typing import Callable

import numpy as np
import torch
import torch.nn.functional as F

# Game and players
from rgi.rgizero.games.connect4 import Connect4Game
from rgi.rgizero.players.alphazero import AlphazeroPlayer
from rgi.rgizero.players.alphazero import play_game

from notebook_utils import reload_local_modules

print("✅ Imports successful")

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
  device = 'mps'
else:
    device = 'cpu'
print(f'Using device: {device}')
assert device in ('cuda', 'mps'), f"No accelerator available, device={device}"

# Allow asyncio to work with jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# Increase numpy print width
np.set_printoptions(linewidth=300)

%load_ext line_profiler

In [ ]:
DEBUG_MODE = True     # Set options to make debugger work properly. Single worker, etc.
LOAD_MODEL = False
TRAIN_MODEL = True
MODEL_SIZE = "small"  # "tiny" or "small" or"large" or "xl"
NUM_SIMULATIONS = 200

## TODO: We should probably concatenate previous trajectories to the current one while training?

# If False, we still load previous games from disk.
NUM_GAMES = 2000
MAX_TRAINING_ITERS = 10000
CONFIG_ALIAS = f'trajectory_sims-{NUM_SIMULATIONS}_games-{NUM_GAMES}_size-{MODEL_SIZE}_train-{MAX_TRAINING_ITERS}'
NUM_GENERATIONS = 5

## Step 1: Set up history-wrapped game


In [ ]:
from rgi.rgizero.games.history_wrapper import HistoryTrackingGame
from rgi.rgizero.data.trajectory_dataset import Vocab
from rgi.rgizero.common import TOKENS

base_game, max_game_length = Connect4Game(connect_length=4), 7*6

game = HistoryTrackingGame(base_game)
state_0 = game.initial_state()
block_size = max_game_length + 2
all_actions = game.all_actions()
action_vocab = Vocab(itos=[TOKENS.START_OF_GAME] + list(all_actions))
n_max_context = max_game_length + 2
game_name = base_game.__class__.__name__

print("✅ Using HistoryTrackingGame from module")
print(f"Game: {game_name}, Players: {game.num_players(state_0)}, Actions: {list(game.all_actions())}")

DATA_DIR = Path.cwd().parent / "data" / "rgizero-e2e" / game_name / CONFIG_ALIAS
print("Creating data dir: ", DATA_DIR)
os.makedirs(DATA_DIR, exist_ok=True)

MODEL_DIR = Path.cwd().parent / "models" / "rgizero-e2e" / game_name / CONFIG_ALIAS
print("Creating model dir: ", MODEL_DIR)
os.makedirs(MODEL_DIR, exist_ok=True)


## Step 2: Create random generation_0 model


In [ ]:
def print_dataset_stats(dataset_path, split_name):
    """Print statistics about a loaded trajectory dataset."""
    td = TrajectoryDataset(dataset_path.parent, split_name, block_size=n_max_context)
    
    # Calculate basic stats
    num_trajectories = len(td)
    total_actions = td._num_actions
    avg_trajectory_length = total_actions / num_trajectories if num_trajectories > 0 else 0
    
    # Get trajectory lengths, winners, and first moves
    trajectory_lengths = []
    winners = []
    first_moves = []
    
    for i in range(num_trajectories):
        start_idx = td.boundaries[i]
        end_idx = td.boundaries[i + 1]
        traj_length = end_idx - start_idx
        trajectory_lengths.append(traj_length)
        
        # Get winner from final values (values are the same throughout trajectory)
        # Values are in range [-1, 1] where positive means player 1 advantage
        final_values = td.value_data[start_idx]  # shape: (num_players,)
        if final_values[0] > final_values[1]:
            winners.append(1)
        elif final_values[1] > final_values[0]:
            winners.append(2)
        else:
            winners.append(None)  # Draw
        
        # Get first move (decode from vocab)
        first_action_encoded = td.action_data[start_idx]
        first_action = action_vocab.decode([first_action_encoded])[0]
        first_moves.append(first_action)
    
    # Print basic stats
    print(f"Dataset Stats:")
    print(f"  Trajectories: {num_trajectories}")
    print(f"  Total actions: {total_actions}")
    print(f"  Avg trajectory length: {avg_trajectory_length:.2f}")
    print(f"  Trajectory length - min: {min(trajectory_lengths)}, max: {max(trajectory_lengths)}, mean: {np.mean(trajectory_lengths):.2f}")
    
    # Print winner stats (similar to print_game_stats)
    print(f"Winner Stats:")
    winner_stats = Counter(winners)
    total_games = num_trajectories
    win1_pct = 100 * winner_stats[1] / total_games if total_games > 0 else 0
    win2_pct = 100 * winner_stats[2] / total_games if total_games > 0 else 0
    print(f"  Winner counts: win[1]={win1_pct:.2f}% win[2]={win2_pct:.2f}%, n={total_games}")
    
    # Print stats by initial move
    print(f"Winner Stats by initial move:")
    move_stats = defaultdict(Counter)
    for first_move, winner in zip(first_moves, winners):
        move_stats[first_move][winner] += 1
    
    for action in sorted(move_stats.keys()):
        counts = move_stats[action]
        total = sum(counts.values())
        win1_pct = 100 * counts[1] / total if total > 0 else 0
        print(f"  a={action}: n={total:3} win[1]={win1_pct:.2f}% counts={counts}")

def print_model_stats(model, config_alias=""):
    """Print statistics about a model."""
    # Count parameters
    num_params = sum(p.numel() for p in model.parameters())
    num_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Model Stats:")
    print(f"  Config: {model.config}")
    print(f"  Total parameters: {num_params:,}")
    print(f"  Trainable parameters: {num_trainable:,}")
    if config_alias:
        print(f"  Config alias: {config_alias}")


In [ ]:
reload_local_modules(verbose=False)

from rgi.rgizero.models.action_history_transformer import ActionHistoryTransformer
from rgi.rgizero.models.transformer import TransformerConfig

model_config_dict = {
    "tiny": TransformerConfig(n_max_context=n_max_context, n_layer=2, n_head=2, n_embd=8),
    "small": TransformerConfig(n_max_context=n_max_context, n_layer=4, n_head=4, n_embd=32),
    "large": TransformerConfig(n_max_context=n_max_context, n_layer=8, n_head=8, n_embd=128),
    "xl": TransformerConfig(n_max_context=n_max_context, n_layer=16, n_head=16, n_embd=256),
}


def create_random_model(config: TransformerConfig, action_vocab_size, num_players,  seed: int):
    torch.manual_seed(seed)
    np.random.seed(seed) # Ensure numpy operations are also seeded
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    model = ActionHistoryTransformer(config=config, action_vocab_size=action_vocab_size, num_players=num_players)
    model.to(device)
    return model

# Make model initialization deterministic
model_config = model_config_dict[MODEL_SIZE]
model_0 = create_random_model(model_config, action_vocab_size=action_vocab.vocab_size, num_players=game.num_players(state_0), seed=42)

# Step 3: Define play & generation code


In [ ]:
from rgi.rgizero.models.action_history_transformer import AsyncNetworkEvaluator, ActionHistoryTransformerEvaluator
from rgi.rgizero.players.alphazero import play_game_async
from tqdm.asyncio import tqdm

async def play_games_async(num_games: int, player_factory: Callable[[], AlphazeroPlayer]):
    tasks = []
    async def create_player_and_create_game():
        t0 = time.time()
        player = player_factory()
        game_result = await play_game_async(game, [player, player])
        t1 = time.time()
        game_result['time'] = t1 - t0
        return game_result

    tasks = [create_player_and_create_game() for _ in range(num_games)]
    results = await tqdm.gather(*tasks)   # same as asyncio.gather, but with a progress bar
    return results

async def play_generation_async(model, num_games, simulations=NUM_SIMULATIONS):
    serial_evaluator = ActionHistoryTransformerEvaluator(model, device=device, block_size=block_size, vocab=action_vocab)
    async_evaluator = AsyncNetworkEvaluator(base_evaluator=serial_evaluator, max_batch_size=1024, verbose=False)

    master_rng = np.random.default_rng(42)
    async_player_factory = lambda: AlphazeroPlayer(game, async_evaluator, rng=np.random.default_rng(master_rng.integers(0, 2**31)), add_noise=False, simulations=simulations)

    await async_evaluator.start()
    results = await play_games_async(num_games=num_games, player_factory=async_player_factory)
    await async_evaluator.stop()
    return results

def print_game_stats(results):
    print("Winner Stats:")
    winner_stats = Counter(result['winner'] for result in results)
    print(f"Winner counts: win[1]={100*winner_stats[1]/sum(winner_stats.values()):.2f}% win[2]={100*winner_stats[2]/sum(winner_stats.values()):.2f}%, n={sum(winner_stats.values())}")
    game_lengths = [len(result['action_history']) for result in results]
    print(f"Game Length min: {min(game_lengths)}, max: {max(game_lengths)}, mean: {np.mean(game_lengths):.2f}")
    print("Winner Stats by initial move:")
    dd = defaultdict(Counter)
    for result in results:
        dd[result['action_history'][0]][result['winner']] += 1
    for action, counts in sorted(dd.items()):
        print(f"  a={action}: n={sum(counts.values()):3} win[1]={100*counts[1]/sum(counts.values()):.2f}% counts={counts}")

## Step 3: Confirm we can read & write to trajectory_dataset


In [ ]:
from rgi.rgizero.data.trajectory_dataset import TrajectoryDatasetBuilder, TrajectoryDataset, build_trajectory_loader
reload_local_modules(verbose=False)

def add_trajectory(game_result, vocab, td_builder):
    action_history = game_result['action_history']
    trajectory_length = len(action_history)
    legal_policies = game_result['legal_policies']
    legal_action_idx = game_result['legal_action_idx']
    rewards = game_result['rewards']

    # Translation key for converting legal_action_ids to vocab_action_idx.
    action_idx_to_vocab_idx = vocab.encode(all_actions)

    fixed_width_policies = np.zeros((trajectory_length, vocab.vocab_size))
    for i in range(trajectory_length):
        vocab_action_idx = action_idx_to_vocab_idx[legal_action_idx[i]]
        fixed_width_policies[i, vocab_action_idx] = legal_policies[i]

    encoded_action_history = vocab.encode(action_history)
    tiled_rewards = np.tile(rewards, (trajectory_length, 1))  # shape (num_players,) -> (num_moves, num_players)
    
    td_builder.add_trajectory(actions=encoded_action_history, fixed_width_policies=fixed_width_policies, values=tiled_rewards)

def write_trajectory_dataset(results, action_vocab, generation_id):
    td_builder = TrajectoryDatasetBuilder(action_vocab)
    for game_result in results:
        add_trajectory(game_result, action_vocab, td_builder)

    trajectory_path = td_builder.save(DATA_DIR, f"gen-{generation_id}")
    return trajectory_path

## Train model


In [ ]:
from rgi.rgizero.train import Trainer, TrainConfig

train_config = TrainConfig(
    model_name="connect4-e2e",
    model_version="v1",

    eval_interval = 250,  # keep frequent because we'll overfit
    eval_iters = 200,
    log_interval = 10_000,  # don't print too too often

    # we expect to overfit on this small dataset, so only save when val improves
    always_save_checkpoint = False,

    gradient_accumulation_steps = 1,
    batch_size = 64,

    learning_rate = 1e-3,  # with baby networks can afford to go a bit higher
    max_iters = MAX_TRAINING_ITERS,
    lr_decay_iters = 5000,  # make equal to max_iters usually
    min_lr = 1e-4,  # learning_rate / 10 usually
    beta2 = 0.99,  # make a bit bigger because number of tokens per iter is small

    warmup_iters = 100,  # not super necessary potentially
)

def train_model(model, split, train_config):
    # Load dataset
    num_workers = 0 if DEBUG_MODE else 4

    td = TrajectoryDataset(DATA_DIR, split, block_size=n_max_context)
    trajectory_loader = build_trajectory_loader(
        DATA_DIR, split, block_size=n_max_context, batch_size=1,
        device=device, workers=num_workers)
        
    trainer = Trainer(
        model=model,
        train_config=train_config,
        train_loader=trajectory_loader,
        val_loader=trajectory_loader,  # TODO: Create separate validation loader
        device=device
    )

    trainer.train()
    return model, trainer

In [ ]:
import dataclasses

def get_model_path(generation_id):
    return MODEL_DIR / f"gen-{generation_id}.pt"

def save_model(model, trainer, generation_id):
    # Save model
    model_path = get_model_path(generation_id)

    checkpoint = {
        'model': model.state_dict(),
        'model_config': dataclasses.asdict(model.config),
        'vocab': action_vocab.to_dict(),
        'iter_num': trainer.iter_num,
        'best_val_loss': trainer.best_val_loss,
        'num_players': game.num_players(state_0),
    }
    torch.save(checkpoint, model_path)
    return model_path

def load_model(generation_id):
    model_path = get_model_path(generation_id)
    loaded_checkpoint = torch.load(model_path)
    loaded_model = ActionHistoryTransformer(
        config=TransformerConfig(**loaded_checkpoint['model_config']),
        action_vocab_size=loaded_checkpoint['vocab']['vocab_size'],
        num_players=loaded_checkpoint['num_players']
    )
    loaded_model.to(device)
    return loaded_model


In [ ]:
# Do a single generation of play & train
async def run_generation(model, num_games, num_simulations, generation_id):
    print(f"\n\n## Running generation {generation_id} for config_alias={CONFIG_ALIAS}")
    split_name = f"gen-{generation_id}"
    expected_trajectory_path = DATA_DIR / split_name
    if not expected_trajectory_path.exists():
        print(f"Playing {num_games} games, simulations={num_simulations}, model_size={MODEL_SIZE}")
        results = await play_generation_async(model, num_games=NUM_GAMES, simulations=NUM_SIMULATIONS)
        print_game_stats(results)
        trajectory_path = write_trajectory_dataset(results, action_vocab, generation_id)
        assert trajectory_path == expected_trajectory_path
    else:
        print(f"Loading trajectory from {expected_trajectory_path}")
        print_dataset_stats(expected_trajectory_path, split_name)
        trajectory_path = expected_trajectory_path
        results = None

    model_path = get_model_path(generation_id)
    if not model_path.exists():
        print(f"Training model on {split_name}")
        updated_model, trainer = train_model(model, split_name, train_config)
        save_model(updated_model, trainer, generation_id)
    else:
        print(f"Loading model from {model_path}")
        updated_model = load_model(generation_id)
        print_model_stats(updated_model, config_alias=MODEL_SIZE)

    return results, trajectory_path, updated_model

In [ ]:
results_dict = {}
trajectory_paths_dict = {}
model_dict = {0: model_0}

for generation_id in range(1, 10):
    current_model = model_dict[generation_id-1]
    results_i, trajectory_path_i, model_i = await run_generation(current_model, num_games=NUM_GAMES, num_simulations=NUM_SIMULATIONS, generation_id=generation_id)
    results_dict[generation_id] = results_i
    trajectory_paths_dict[generation_id] = trajectory_path_i
    model_dict[generation_id] = model_i
